In [18]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [19]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [20]:
conn = sqlite3.connect(DATABASE_PATH)

precip_monthly = pd.read_sql(sql='SELECT * FROM gee_precip_monthly', con=conn)
precip_monthly['Date'] = pd.to_datetime(precip_monthly['Date'])

pet_monthly = pd.read_sql(sql='SELECT * FROM gee_pet_monthly', con=conn)
pet_monthly['Date'] = pd.to_datetime(pet_monthly['Date'])

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [21]:
precip_monthly

,Station_ID,Date,ERA5_Precipitation,GPM_Precipitation,TRMM_Precipitation,TERRACLIMATE_Precipitation,PERSIANNCDR_Precipitation
0,40709,2006-09-30,130.801336,115.920010,54.357873,117.0,39.081124
1,40709,2006-10-31,280.789058,164.424012,120.467601,212.0,84.028451
2,40709,2006-11-30,245.921390,101.520002,67.297440,122.0,78.084248
3,40709,2006-12-31,183.809788,77.376002,74.464657,110.0,76.726466
4,40709,2007-01-31,64.469668,26.040000,5.355088,26.0,21.264424
...,...,...,...,...,...,...,...
5131,99361,2024-02-29,100.628399,82.824002,NaN,NaN,49.011868
5132,99361,2024-03-31,80.836829,16.368001,NaN,NaN,24.582743
5133,99361,2024-04-30,112.907503,36.000001,NaN,NaN,NaN
5134,99361,2024-05-31,191.929751,52.080000,NaN,NaN,NaN


In [22]:
pet_monthly

,Station_ID,Date,PET_MOD16A2GF
0,40709,2006-09-30,125.7250
1,40709,2006-10-31,80.7750
2,40709,2006-11-30,44.0000
3,40709,2006-12-31,35.0250
4,40709,2007-01-31,60.4000
...,...,...,...
4987,99361,2023-08-31,239.2750
4988,99361,2023-09-30,176.6500
4989,99361,2023-10-31,123.0125
4990,99361,2023-11-30,98.8000


In [23]:
geoinfo

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation
0,40759,Sari,Mazandaran,36.536,52.998,23.0
1,99306,Bandar-e-amirabad,Mazandaran,36.856,53.386,-20.0
2,99357,Baladeh,Mazandaran,36.198,51.801,2120.0
3,99299,Galugah,Mazandaran,36.738,53.837,-10.0
4,40737,Gharakhil,Mazandaran,36.487,52.108,14.7
5,40760,Kiyasar,Mazandaran,36.248,53.546,1294.3
6,99361,Alasht,Mazandaran,36.071,52.843,1805.0
7,99309,Amol,Mazandaran,36.479,52.468,23.7
8,99348,Kojur,Mazandaran,36.390,51.729,1550.0
9,99360,Polsefid,Mazandaran,36.104,53.062,610.0


In [24]:
precip_models = ['ERA5', 'GPM', 'TRMM', 'TERRACLIMATE', 'PERSIANNCDR']
pet_model = 'MOD16A2GF'

data_si = precip_monthly\
    .merge(pet_monthly, on=['Station_ID', 'Date'], how='outer')\
    .set_index(['Date'])

for pm in precip_models:
    data_si[f'PE_{pm.split('_')[0]}'] = data_si[f'{pm}_Precipitation'] - data_si[f'PET_{pet_model}']

timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
i = 1

for ts in timescale:
    for pm in precip_models: 
        df_spi = data_si\
            .groupby(by='Station_ID')\
            .apply(
                lambda x: spei.spi(
                    series=x[f'{pm}_Precipitation'],
                    dist=scs.gamma,
                    prob_zero=True,
                    timescale=ts
                ),
                include_groups=False
            )\
            .reset_index()
        df_spi = df_spi.melt(id_vars="Station_ID", value_name=f'{pm}_SPI_{ts}')
        if i == 1:
            results = df_spi.copy()
        else:
            results = results.merge(df_spi, on=['Station_ID', 'Date'], how='outer')
        
        df_spei = data_si\
            .groupby(by='Station_ID')\
            .apply(
                lambda x: spei.spei(
                    series=x[f'PE_{pm}'],
                    dist=scs.fisk,
                    prob_zero=True,
                    timescale=ts
                ),
                include_groups=False
            )\
            .reset_index()
        df_spei = df_spei.melt(id_vars="Station_ID", value_name=f'{pm}_SPEI_{ts}')
        results = results.merge(df_spei, on=['Station_ID', 'Date'], how='outer')

        i += 1


results.info()

/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:1190: RuntimeWarning: overflow encountered in power
  - sc.xlog1py(d_+1, x_**(-c_))))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 92 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Station_ID            5136 non-null   int64  
 1   Date                  5136 non-null   object 
 2   ERA5_SPI_1            5136 non-null   float64
 3   ERA5_SPEI_1           4992 non-null   float64
 4   GPM_SPI_1             5136 non-null   float64
 5   GPM_SPEI_1            4992 non-null   float64
 6   TRMM_SPI_1            3840 non-null   float64
 7   TRMM_SPEI_1           3840 non-null   float64
 8   TERRACLIMATE_SPI_1    4992 non-null   float64
 9   TERRACLIMATE_SPEI_1   4992 non-null   float64
 10  PERSIANNCDR_SPI_1     5064 non-null   float64
 11  PERSIANNCDR_SPEI_1    4992 non-null   float64
 12  ERA5_SPI_3            5088 non-null   float64
 13  ERA5_SPEI_3           4944 non-null   float64
 14  GPM_SPI_3             5088 non-null   float64
 15  GPM_SPEI_3           

In [27]:
results['Date'] = pd.to_datetime(results['Date'])
results

,Station_ID,Date,ERA5_SPI_1,ERA5_SPEI_1,GPM_SPI_1,GPM_SPEI_1,TRMM_SPI_1,TRMM_SPEI_1,TERRACLIMATE_SPI_1,TERRACLIMATE_SPEI_1,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TERRACLIMATE_SPI_24,TERRACLIMATE_SPEI_24,PERSIANNCDR_SPI_24,PERSIANNCDR_SPEI_24
0,40709,2006-09-30,-0.002205,0.016456,0.308333,0.233141,-0.386448,-0.537595,0.310259,0.306610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40709,2006-10-31,0.495160,0.453269,0.657774,0.310706,0.413572,0.040171,1.474084,0.760626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40709,2006-11-30,0.392122,0.432805,-0.170851,-0.113445,-0.519774,-0.488849,0.245867,0.252009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40709,2006-12-31,0.446678,0.477084,-0.205610,-0.095285,0.250047,0.401613,0.567163,0.646242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40709,2007-01-31,-1.201812,-1.212942,-1.996864,-1.773098,-1.835450,-1.771789,-1.746711,-1.608924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,99361,2024-02-29,0.696967,NaN,0.794496,NaN,NaN,NaN,NaN,NaN,...,-1.494781,NaN,-0.884397,NaN,NaN,NaN,NaN,NaN,-5.877000,NaN
5132,99361,2024-03-31,-0.969038,NaN,-1.917287,NaN,NaN,NaN,NaN,NaN,...,-1.630171,NaN,-1.943880,NaN,NaN,NaN,NaN,NaN,-6.136069,NaN
5133,99361,2024-04-30,0.188096,NaN,0.005853,NaN,NaN,NaN,NaN,NaN,...,-1.800329,NaN,-1.772096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,99361,2024-05-31,1.390493,NaN,1.492357,NaN,NaN,NaN,NaN,NaN,...,-1.602138,NaN,-0.694518,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('gee_di_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()